# Задатак класификације текста

Као што смо поменули, фокусираћемо се на једноставан задатак класификације текста заснован на **AG_NEWS** скупу података, који подразумева класификацију наслова вести у једну од 4 категорије: Свет, Спорт, Бизнис и Наука/Технологија.

## Скуп података

Овај скуп података је уграђен у модул [`torchtext`](https://github.com/pytorch/text), тако да му можемо лако приступити.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Овде, `train_dataset` и `test_dataset` садрже збирке које враћају парове ознаке (број класе) и текста, на пример:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Дакле, хајде да одштампамо првих 10 нових наслова из нашег скупа података:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Пошто су скупови података итератори, ако желимо да користимо податке више пута, морамо их претворити у листу:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Токенизација

Сада треба да претворимо текст у **бројеве** који могу бити представљени као тензори. Ако желимо представљање на нивоу речи, потребно је да урадимо две ствари:
* користимо **токенизатор** за раздвајање текста на **токене**
* направимо **речник** тих токена.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Коришћењем речника, можемо лако кодирати наш токенизовани низ у скуп бројева:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Представљање текста помоћу вреће речи

Пошто речи носе значење, понекад можемо разумети значење текста само гледајући појединачне речи, без обзира на њихов редослед у реченици. На пример, при класификацији вести, речи као што су *време*, *снег* вероватно указују на *временску прогнозу*, док речи као што су *акције*, *долар* могу указивати на *финансијске вести*.

**Врећа речи** (BoW) представљање у виду вектора је најчешће коришћено традиционално представљање вектора. Свака реч је повезана са индексом вектора, а елемент вектора садржи број појављивања те речи у датом документу.

![Слика која приказује како је представљање вектора вреће речи приказано у меморији.](../../../../../translated_images/sr/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Напомена**: Можете такође размишљати о BoW као о збиру свих вектора кодираних једним битом за појединачне речи у тексту.

Испод је пример како да генеришете представљање вреће речи користећи Scikit Learn библиотеку за Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Да бисмо израчунали вектор торбе речи из векторске репрезентације нашег AG_NEWS скупа података, можемо користити следећу функцију:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Напомена:** Овде користимо глобалну променљиву `vocab_size` да бисмо одредили подразумевану величину речника. Пошто је величина речника често прилично велика, можемо ограничити величину речника на најчешће речи. Покушајте да смањите вредност `vocab_size` и покренете код испод, и видите како то утиче на тачност. Требало би да очекујете одређени пад тачности, али не драматичан, у замену за боље перформансе.


## Тренирање BoW класификатора

Сада када смо научили како да направимо представу текста помоћу Bag-of-Words, хајде да обучимо класификатор на основу тога. Прво, потребно је да конвертујемо наш скуп података за тренирање тако да све позиционе векторске представе буду претворене у Bag-of-Words представу. Ово можемо постићи прослеђивањем функције `bowify` као параметра `collate_fn` стандардном torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Сада хајде да дефинишемо једноставну класификаторску неуронску мрежу која садржи један линеарни слој. Величина улазног вектора је једнака `vocab_size`, а величина излазног одговара броју класа (4). Пошто решавамо задатак класификације, завршна активациона функција је `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Сада ћемо дефинисати стандардну PyTorch петљу за тренирање. Пошто је наш скуп података прилично велик, за потребе нашег учења тренираћемо само један епох, а понекад чак и мање од једног епоха (параметар `epoch_size` нам омогућава да ограничимо тренирање). Такође ћемо пријављивати акумулирану тачност тренирања током тренирања; учесталост пријављивања се одређује помоћу параметра `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## Биграми, Триграми и Н-грами

Једно ограничење приступа "вреће речи" је то што су неке речи део израза који се састоје од више речи. На пример, реч „хот дог“ има потпуно другачије значење од речи „хот“ и „дог“ у другим контекстима. Ако речи „хот“ и „дог“ увек представљамо истим векторима, то може збунити наш модел.

Да бисмо решили овај проблем, **Н-грам репрезентације** се често користе у методама класификације докумената, где је учесталост сваке речи, двосложног или тросложног израза корисна карактеристика за тренирање класификатора. У биграм репрезентацији, на пример, додаћемо све парове речи у речник, поред оригиналних речи.

Испод је пример како да генеришете биграм репрезентацију „вреће речи“ користећи Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Главни недостатак N-gram приступа је што величина речника почиње да расте изузетно брзо. У пракси, потребно је комбиновати N-gram репрезентацију са неким техникама за смањење димензионалности, као што су *уграђивања* (*embeddings*), о којима ћемо говорити у наредној јединици.

Да бисмо користили N-gram репрезентацију у нашем **AG News** скупу података, потребно је да изградимо посебан ngram речник:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Могли бисмо користити исти код као горе за тренирање класификатора, међутим, то би било веома неефикасно у погледу меморије. У наредној јединици, тренираћемо класификатор са биграмима користећи ембедингсе.

> **Напомена:** Можете оставити само оне нграме који се у тексту појављују више од одређеног броја пута. Ово ће осигурати да се ретки биграми изоставе и значајно смањити димензионалност. Да бисте то урадили, подесите параметар `min_freq` на већу вредност и посматрајте промену дужине речника.


## Учестаност термина и обрнута учестаност докумената TF-IDF

У представљању BoW, појаве речи се равномерно вреднују, без обзира на саму реч. Међутим, јасно је да су учестале речи, као што су *a*, *in*, итд., много мање важне за класификацију у поређењу са специјализованим терминима. У ствари, у већини NLP задатака неке речи су значајније од других.

**TF-IDF** означава **учестаност термина–обрнута учестаност докумената**. То је варијација модела торбе речи, где се уместо бинарне вредности 0/1 која указује на појаву речи у документу, користи вредност са покретним зарезом, која је повезана са учестаношћу појаве речи у корпусу.

Формалније, тежина $w_{ij}$ речи $i$ у документу $j$ дефинише се као:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
где
* $tf_{ij}$ представља број појављивања $i$ у $j$, односно BoW вредност коју смо раније видели
* $N$ је број докумената у збирци
* $df_i$ је број докумената који садрже реч $i$ у целој збирци

TF-IDF вредност $w_{ij}$ расте пропорционално броју пута када се реч појави у документу и смањује се у зависности од броја докумената у корпусу који садрже ту реч, што помаже да се прилагоди чињеници да се неке речи чешће појављују од других. На пример, ако се реч појављује у *сваком* документу у збирци, $df_i=N$, и $w_{ij}=0$, те би ти термини били потпуно занемарени.

TF-IDF векторизацију текста можете лако креирати користећи Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Закључак

Иако TF-IDF репрезентације дају тежину учесталости различитим речима, оне нису у могућности да представе значење или редослед. Као што је чувени лингвиста Џ. Р. Фирт рекао 1935. године: „Потпуно значење речи је увек контекстуално, и ниједно проучавање значења ван контекста не може се сматрати озбиљним.” У наставку курса ћемо научити како да ухватимо контекстуалне информације из текста користећи језичко моделирање.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
